In [2]:
# ==whale_recommend_task3-2-FM==

# 5.1FM模型的引入
# 5.1.1逻辑回归模型及其缺点
# CTR预估，即广告点击预估，一个用户是点还是不点。思路一般用就是将特征做线性组合（逻辑回归
# LR），传入sigmoid中得到一个概率值，本质上这就是一个线性模型。但效果较差。
# 不会改变CTR预测顺序；
# 线性模型；
# 每个特征对最终输出结果独立，需要手动特征交叉（ ），比较麻烦

# 5.1.2二阶交叉项的考虑及改进
# 手动做特征交叉比较麻烦，干脆就考虑所有的二阶交叉项，y= b+wx 后面加了wijxixj加总。
# 但这有一个问题，只有当xi与xj均不为0时这个二阶交叉项才会生效，后面这个特征交叉项本质是和多项式核
# SVM等价的。
# 为了解决上面问题FM出现了。改动处：将上式的wij换成了<vi,vj>,
# 意思是，给每个xi计算一个embedding，然后将两个向量之间的embedding做内积，起到Wij的作用。
# 这样两个定称不同时出现的话，也不会计算不出wij，泛化能力增强。

# 5.2FM公式的理解

# 根据定理任意一个实对称矩阵W都存在一个V，使得W = V.V^T。
# 所以，可以将W分解，得到两个隐向量（详见上一章，用于关系用户与商品，训练隐含关系，关联预测未知用户对商品的评价，做推荐）
# FM的公式是一个通用的拟合方程，可以采用不同的损失函数用于解决regression、classification等问题，比如可以采用MSE（Mean Square Error）loss function来求解回归问题，也可以采用Hinge/Cross-Entropy loss来求解分类问题。当然，在进行二元分类时，FM的输出需要使用sigmoid函数进行变换，该原理与LR是一样的。

# 5.3 FM模型的应用

# 最直接的想法就是直接把FM得到的结果放进sigmoid中输出一个概率值，由此做CTR预估，事实上我们也可以做召回。
# 由于FM模型是利用两个特征的Embedding做内积得到二阶特征交叉的权重，那么我们可以将训练好的FM特征取出离线存好，之后用来做KNN向量检索。

# 工业应用的具体操作步骤：
# 1. 离线训练好FM模型（学习目标可以是CTR） 
# 2. 将训练好的FM模型Embedding取出
# 3. 将每个uid对应的Embedding做avg pooling（平均）形成该用户最终的Embedding，item也做同样的操作
# 4. 将所有的Embedding向量放入Faiss（为稠密向量提供高效相似度搜索和聚类的框架）等 
# 5. 线上uid发出请求，取出对应的user embedding，进行检索召回

In [ ]:
# 5.4代码实践

#一、安装 fm，导包
# pip install git+https://github.com/coreylynch/pyFM
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [6]:
# 二、创建训练集并转换成one-hot编码的特征形式
train = [
 {"user": "1", "item": "5", "age": 19},
 {"user": "2", "item": "43", "age": 33},
 {"user": "3", "item": "20", "age": 55},
 {"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)
print(X.toarray())

[[19.  0.  0.  0.  1.  1.  0.  0.  0.]
 [33.  0.  0.  1.  0.  0.  1.  0.  0.]
 [55.  0.  1.  0.  0.  0.  0.  1.  0.]
 [20.  1.  0.  0.  0.  0.  0.  0.  1.]]


In [7]:
# 用、创建标签，全1的，简单点
y = np.repeat(1.0,X.shape[0])
y

array([1., 1., 1., 1.])

In [ ]:
# 四、训练并预测
fm = pylibfm.FM()
fm.fit(X,y)
fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))

In [ ]:
# 5.4.1.1电影评分数据集实战
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

In [ ]:
#读数据
def loadData(filename,path="ml-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
    for line in f:
        (user,movieid,rating,ts)=line.split('\t')
        data.append({ "user_id": str(user), "movie_id": str(movieid)})
        y.append(float(rating))
        users.add(user)
        items.add(movieid)
    return (data, np.array(y), users, items)

In [ ]:
# 导入训练集和测试集，并转换格式
(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")
v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

In [ ]:
# 训练模型并测试
# Build and train a Factorization Machine
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", 
initial_learning_rate=0.001, learning_rate_schedule="optimal")
fm.fit(X_train,y_train)

In [ ]:
# 预测结果打印误差
preds = fm.predict(X_test)
from sklearn.metrics import mean_squared_error
print("FM MSE: %.4f" % mean_squared_error(y_test,preds))
# FM MSE: 0.8873

In [ ]:
# 5.4.1.2分类任务实战

import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from pyfm import pylibfm
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=1000,n_features=100, n_clusters_per_class=1)
data = [ {v: k for k, v in dict(zip(i, range(len(i)))).items()} for i in X]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=42)
v = DictVectorizer()
X_train = v.fit_transform(X_train)
X_test = v.transform(X_test)

In [ ]:
# 建模型，与上例比，主要改变的参数是 num_factors 和 tasks
fm = pylibfm.FM(num_factors=50, num_iter=10, verbose=True, task="classification", 
initial_learning_rate=0.0001, learning_rate_schedule="optimal")
fm.fit(X_train,y_train)


In [ ]:
# 由于是分类任务，误差函数肯定不一样
from sklearn.metrics import log_loss
print("Validation log loss: %.4f" % log_loss(y_test,fm.predict(X_test)))
# Validation log loss: 1.3678

In [12]:
# 5.4.2从零实现
# 数据集介绍，criteo：criteo是非常经典的点击率预估数据集，其中连续特征有13个，类别型特征有26个，没有提供特征的具体名称，特
# 征分别如下：
# dense_feats：'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10','I11', 'I12', 'I13'
# sparse_feats: 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 
# 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'
# 代码参考FM.py，待更新


In [ ]:
# 课后问题：FM存在的问题， 以及可以从哪些地方再进行改进？